In [1]:
print("start")

### google.colab 掛載雲端硬碟(google.colab才需要執行)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !ls 'drive'

In [4]:
# import os
# path = "/content/drive/My Drive/Colab Notebooks/qagnn/reference/"
# os.chdir(path)

In [5]:
!ls

 LICENSE
'QA-GNN Reasoning with Language Models and Knowledge Graphs for Question Answering.pdf'
 README.md
 data
 data_preprocessed_release.zip
 download_preprocessed_data.sh
 download_raw_data.sh
 eval.ipynb
 figs
 modeling
 qagnn.py
 run_qagnn__csqa.sh
 run_qagnn__obqa.sh
 saved_models
 train.ipynb
 train_csqa__enc-roberta-large__k5__gnndim200__bs128__seed0__20211009_182958.log.txt
 utils
 wgetDataPreprocessed.ipynb


## 安裝套件與環境

In [6]:
#conda install pytorch==1.4.0 torchvision==0.5.0 cudatoolkit=10.1 -c pytorch
!pip install torch==1.4.0 torchvision==0.5.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import torch

In [8]:
print(torch.__version__)

1.4.0


In [9]:
print(torch.version.cuda)

10.1


In [10]:
!pip install numpy==1.18.3 tqdm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
!pip install transformers==2.0.0 nltk spacy==2.1.6

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [12]:
!python -m spacy download en

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
/user_data/.local/lib/python3.6/site-packages/en_core_web_sm -->
/user_data/.local/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [13]:
!pip install torch-scatter==2.0.4 -f https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
!pip install torch-cluster==1.5.4 -f https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [15]:
!pip install torch-sparse==0.6.1 -f https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
!pip install torch-spline-conv==1.2.0 -f https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [17]:
!pip install torch-geometric==1.6.0 -f https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0+cu101.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## 執行訓練

In [18]:
# anaconda用%%shell跑shell
#%%shell
# 實驗室環境用%%bash跑shell
#%%bash

In [19]:
%%bash

export PYTHONIOENCODING=utf-8
export CUDA_VISIBLE_DEVICES=0,1
dt=`date '+%Y%m%d_%H%M%S'`


dataset="csqa"
model='roberta-large'
shift
shift
args=$@


elr="1e-5"
dlr="1e-3"
bs=128
n_epochs=4

k=5 #num of gnn layers
gnndim=200

echo "***** hyperparameters *****"
echo "dataset: $dataset"
echo "enc_name: $model"
echo "batch_size: $bs"
echo "learning_rate: elr $elr dlr $dlr"
echo "gnn: dim $gnndim layer $k"
echo "******************************"

save_dir_pref='saved_models'
mkdir -p $save_dir_pref

###### Training ######
for seed in 0; do
  python3 -u qagnn.py --dataset $dataset \
      --encoder $model -k $k --gnn_dim $gnndim -elr $elr -dlr $dlr -bs $bs --seed $seed \
      --n_epochs $n_epochs --max_epochs_before_stop 10  \
      --train_adj data/${dataset}/graph/train.graph.adj.pk \
      --dev_adj   data/${dataset}/graph/dev.graph.adj.pk \
      --test_adj  data/${dataset}/graph/test.graph.adj.pk \
      --train_statements  data/${dataset}/statement/train.statement.jsonl \
      --dev_statements  data/${dataset}/statement/dev.statement.jsonl \
      --test_statements  data/${dataset}/statement/test.statement.jsonl \
      --save_model \
      --save_dir ${save_dir_pref}/${dataset}/enc-${model}__k${k}__gnndim${gnndim}__bs${bs}__seed${seed}__${dt} $args \
  > train_${dataset}__enc-${model}__k${k}__gnndim${gnndim}__bs${bs}__seed${seed}__${dt}.log.txt
done

***** hyperparameters *****
dataset: csqa
enc_name: roberta-large
batch_size: 128
learning_rate: elr 1e-5 dlr 1e-3
gnn: dim 200 layer 5
******************************


2021-10-10 00:51:36.072624: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/user_data/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/user_data/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
100%|������������������������������| 621/62

In [20]:
print("end")

end
